
# Use pipeline 1 data cleansing-project folder

In [ ]:
# Uploads the csv file to colab uncomment two lines below to upload;
from google.colab import files
uploaded = files.upload()

Saving kym.json to kym.json


In [ ]:
#!pip install py2neo -q
#from py2neo.data import Node, Relationship
# https://py2neo.org/v4/data.html can read on how to use it here

import json
import time

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import numpy as np
# !pip install openpyxl
# import openpyxl 
import datetime
from functools import reduce

In [ ]:
with open('kym.json','r') as f:
  data = json.loads(f.read())
  df0 = pd.json_normalize(data,max_level=0) # content, additional references need this level!
  df2 = pd.json_normalize(data,max_level=2)
  columns0 = ['title','url','last_update_source','category','template_image_url','added','tags','search_keywords','parent','additional_references']
  columns2 = [
      'meta.og:image:width','meta.og:image:height','meta.og:description',
      'details.status','details.origin','details.year','details.type',
      'content.about.text','content.about.images','content.about.links','content.origin.text','content.origin.images','content.origin.links',
      'content.spread.text','content.spread.images','content.spread.links','content.notable examples.text','content.notable examples.images','content.notable examples.links',
      'content.search interest.text','content.search interest.images','content.search interest.links','content.external references.text','content.external references.links']
  df0 = df0[columns0]
  df2 = df2[columns2]
  df = pd.concat([df0,df2],axis=1)
  df.columns = ['Title','URL','TimeUpdated','Category','Image','TimeAdded','Tags','Keywords','Parent','References',
              'ImageWidth','ImageHeight','SocialMediaDescription','Status','Origin','Year','Type',
              'AboutText', 'AboutImages', 'AboutLinks', 'OriginText', 'OriginImages', 'OriginLinks',
              'SpreadText', 'SpreadImages', 'SpreadLinks', 'NotExamplesText', 'NotExamplesImages', 'NotExamplesLinks',
              'SearchIntText', 'SearchIntImages', 'SearchIntLinks', 'ExtRefText', 'ExtRefLinks']

In [ ]:
df = df[df.Category=="Meme"]
df = df.drop_duplicates(subset='Title', keep='first')
df = df.drop("Category",axis=1)

In [ ]:
df.head(10)

,Title,URL,TimeUpdated,Image,TimeAdded,Tags,Keywords,Parent,References,ImageWidth,ImageHeight,SocialMediaDescription,Status,Origin,Year,Type,AboutText,AboutImages,AboutLinks,OriginText,OriginImages,OriginLinks,SpreadText,SpreadImages,SpreadLinks,NotExamplesText,NotExamplesImages,NotExamplesLinks,SearchIntText,SearchIntImages,SearchIntLinks,ExtRefText,ExtRefLinks
0,This is Relevant To My Interests,https://knowyourmeme.com/memes/this-is-relevan...,1547002898,https://i.kym-cdn.com/entries/icons/original/0...,1.229113e+09,"[image macros, comment, comments, approval, im...","[\""This is relevant to my interests\""]",NaN,{},600,315,"""This is relevant to my interests"" is a popula...",confirmed,I Can Has Cheezburger,2006,NaN,"[""This is Relevant To My Interests"" is an expr...",NaN,"[[image macros, https://knowyourmeme.com/memes...",[While the phrase is rumored to have originate...,NaN,"[[4chan, https://knowyourmeme.com/memes/sites/...","[On November 16th, 2006, Flickr[3] user ""the b...",[https://i.kym-cdn.com/photos/images/newsfeed/...,"[[LOLcat, https://knowyourmeme.com/memes/lolca...",NaN,[https://i.kym-cdn.com/photos/images/newsfeed/...,NaN,"[Search query volume for the keywords ""this is...",NaN,"[[Caturday, https://knowyourmeme.com/memes/cat...",[[1] I Can Has Cheezburger – This is relevant ...,"[[This is relevant to my interests, http://ica..."
1,ROFLcopter,https://knowyourmeme.com/memes/roflcopter,1591400337,https://i.kym-cdn.com/entries/icons/original/0...,1.229113e+09,"[ascii, animated gif, rofl, acronym, initialism]",[roflcopter],https://knowyourmeme.com/memes/lol-ascii,{'Encyclopedia Dramatica': 'https://encycloped...,600,315,An ASCII helicopter becomes an Internet meme. ...,confirmed,Warcraft III,2004,[https://knowyourmeme.com/types/slang],[ROFLcopter is an Internet slang term used as ...,NaN,"[[Lollerskates, https://knowyourmeme.com/memes...",[According to an Urban Dictionary[1] submissio...,[https://i.kym-cdn.com/photos/images/newsfeed/...,"[[Urban Dictionary, https://knowyourmeme.com/m...","[On March 30th, 2004, the single-serving site ...",[https://i.kym-cdn.com/photos/images/newsfeed/...,"[[single-serving site, https://knowyourmeme.co...",NaN,NaN,NaN,"[By September of 2005, searches for the term s...",NaN,NaN,"[[1] Urban Dictionary – roflcopter, [2] Wiktio...","[[roflcopter, https://www.urbandictionary.com/..."
2,Bitches Don't Know,https://knowyourmeme.com/memes/bitches-dont-know,1607986999,https://i.kym-cdn.com/entries/icons/original/0...,1.229113e+09,"[myspace, 4chan, parody, know, bitches, dont, ...",[bitches don't know],NaN,"{'Facebook,': 'https://www.facebook.com/search...",600,315,A braggardly young man flaunts his genitals vi...,confirmed,MySpace,2006,[https://knowyourmeme.com/types/exploitable],[Bitches Don't Know is an exploitable image ma...,NaN,"[[exploitable, https://knowyourmeme.com/memes/...",[The picture was originally a Myspace photo of...,[https://i.kym-cdn.com/photos/images/newsfeed/...,"[[Myspace, https://knowyourmeme.com/memes/site...","[One of the earliest archived instances of ""bi...",NaN,"[[4chan, https://knowyourmeme.com/memes/sites/...",NaN,[https://i.kym-cdn.com/photos/images/newsfeed/...,NaN,"[""Bitches Don't Know"" saw a viral takeoff in l...",NaN,NaN,[[1] Myspace (via Wayback Machine) – Rich Owen...,"[[Rich Owens, https://web.archive.org/web/2010..."
3,Leave Britney Alone,https://knowyourmeme.com/memes/leave-britney-a...,1613129339,https://i.kym-cdn.com/entries/icons/original/0...,1.229113e+09,"[youtube, rant, britney spears, response, frea...","[\""leave britney alone\"", \""chris crocker\""]",https://knowyourmeme.com/memes/people/chris-cr...,{'Wikipedia': 'https://en.wikipedia.org/wiki/C...,600,315,“Leave Britney Alone” is a viral video in whic...,confirmed,YouTube,2007,NaN,"[""Leave Britney Alone"" is a viral video in whi...",NaN,"[[viral video, https://knowyourmeme.com/memes/...","[On September 10, 2007, Chris Crocker uploaded...",NaN,"[[MTV Music Awards, https://knowyourmeme.com/

In [ ]:
def format_time(col_name):
   formatted_date_time=pd.to_datetime(col_name.apply(datetime.datetime.fromtimestamp))
   return formatted_date_time

def repair_string(df, col_name):
    df[col_name] = df[col_name].astype("str")
    df[col_name] = df.apply(lambda row: row[col_name].replace('"','').replace("'",""),axis=1)
    return df[col_name]

def format_fields(df):
    df["Title"] = df["Title"].astype("str")
    df["URL"] = df["URL"].astype("str")
    # time:
    df['TimeUpdated'] = df['TimeUpdated'].fillna(-2208988800)
    df['TimeUpdated'] = format_time(df['TimeUpdated'])
    df['TimeAdded'] = df['TimeAdded'].fillna(-2208988800)
    df['TimeAdded'] = format_time(df['TimeAdded'])
    # int:
    df['ImageWidth'] = df['ImageWidth'].fillna(0).astype("int64")
    df['ImageHeight'] = df['ImageHeight'].fillna(0).astype("int64")
    df['Year'] = df['Year'].fillna(1900).astype("int64")
    # remove " and ' from string objects:
    columns = ["Title","URL","Image","Tags","Keywords","Parent","References","SocialMediaDescription",
               "Status","Origin","Type","AboutText","AboutImages","AboutLinks","OriginText","OriginImages",
               "OriginLinks","SpreadText","SpreadImages","SpreadLinks","NotExamplesText",
               "NotExamplesImages","NotExamplesLinks","SearchIntText","SearchIntImages","SearchIntLinks",
               "ExtRefText","ExtRefLinks"]
    for col in columns: df[col] = repair_string(df,col)

format_fields(df)

In [ ]:
def tags(df):
    # create intermediate table Tags_exploded 
    # where the initial column Tags is exploded into pieces and saved into new column Tag
    Tags_exploded = df[['URL','Tags']].copy()
    Tags_exploded["Tags"] = Tags_exploded["Tags"].astype('str')
    t1 = Tags_exploded.apply(lambda row: row["Tags"].split('[')[-1], axis=1)
    Tags_exploded["t1"]=t1
    t2 = Tags_exploded.apply(lambda row: row["t1"].split(']')[0], axis=1)
    Tags_exploded["t2"]=t2
    SingleTag = Tags_exploded.apply(lambda row: row["t2"].split(','), axis=1).explode()
    Tags_exploded = Tags_exploded.join(pd.DataFrame(SingleTag,columns=["Tag"]))
    Tags_exploded = Tags_exploded.drop(["Tags","t1","t2"],axis=1)
    return Tags_exploded

def _remove_nsfw(df):
    Tags_exploded = tags(df)
    values = list(Tags_exploded[Tags_exploded.Tag.str.contains('nsfw')].index)
    # drop rows that contain nsfw tag
    df = df[df.index.isin(values) == False]

_remove_nsfw(df)

In [ ]:
df.head(5)


,Title,URL,TimeUpdated,Image,TimeAdded,Tags,Keywords,Parent,References,ImageWidth,ImageHeight,SocialMediaDescription,Status,Origin,Year,Type,AboutText,AboutImages,AboutLinks,OriginText,OriginImages,OriginLinks,SpreadText,SpreadImages,SpreadLinks,NotExamplesText,NotExamplesImages,NotExamplesLinks,SearchIntText,SearchIntImages,SearchIntLinks,ExtRefText,ExtRefLinks
0,This is Relevant To My Interests,https://knowyourmeme.com/memes/this-is-relevan...,2019-01-09 03:01:38,https://i.kym-cdn.com/entries/icons/original/0...,2008-12-12 20:12:41,"[image macros, comment, comments, approval, im...",[\\This is relevant to my interests\\],nan,{},600,315,This is relevant to my interests is a popular ...,confirmed,I Can Has Cheezburger,2006,nan,[This is Relevant To My Interests is an expres...,nan,"[[image macros, https://knowyourmeme.com/memes...",[While the phrase is rumored to have originate...,nan,"[[4chan, https://knowyourmeme.com/memes/sites/...","[On November 16th, 2006, Flickr[3] user the bo...",[https://i.kym-cdn.com/photos/images/newsfeed/...,"[[LOLcat, https://knowyourmeme.com/memes/lolca...",nan,[https://i.kym-cdn.com/photos/images/newsfeed/...,nan,[Search query volume for the keywords this is ...,nan,"[[Caturday, https://knowyourmeme.com/memes/cat...",[[1] I Can Has Cheezburger – This is relevant ...,"[[This is relevant to my interests, http://ica..."
1,ROFLcopter,https://knowyourmeme.com/memes/roflcopter,2020-06-05 23:38:57,https://i.kym-cdn.com/entries/icons/original/0...,2008-12-12 20:13:27,"[ascii, animated gif, rofl, acronym, initialism]",[roflcopter],https://knowyourmeme.com/memes/lol-ascii,{Encyclopedia Dramatica: https://encyclopediad...,600,315,An ASCII helicopter becomes an Internet meme. ...,confirmed,Warcraft III,2004,[https://knowyourmeme.com/types/slang],[ROFLcopter is an Internet slang term used as ...,nan,"[[Lollerskates, https://knowyourmeme.com/memes...",[According to an Urban Dictionary[1] submissio...,[https://i.kym-cdn.com/photos/images/newsfeed/...,"[[Urban Dictionary, https://knowyourmeme.com/m...","[On March 30th, 2004, the single-serving site ...",[https://i.kym-cdn.com/photos/images/newsfeed/...,"[[single-serving site, https://knowyourmeme.co...",nan,nan,nan,"[By September of 2005, searches for the term s...",nan,nan,"[[1] Urban Dictionary – roflcopter, [2] Wiktio...","[[roflcopter, https://www.urbandictionary.com/..."
2,Bitches Dont Know,https://knowyourmeme.com/memes/bitches-dont-know,2020-12-14 23:03:19,https://i.kym-cdn.com/entries/icons/original/0...,2008-12-12 20:13:01,"[myspace, 4chan, parody, know, bitches, dont, ...",[bitches dont know],nan,"{Facebook,: https://www.facebook.com/search/to...",600,315,A braggardly young man flaunts his genitals vi...,confirmed,MySpace,2006,[https://knowyourmeme.com/types/exploitable],[Bitches Don\t Know is an exploitable image ma...,nan,"[[exploitable, https://knowyourmeme.com/memes/...",[The picture was originally a Myspace photo of...,[https://i.kym-cdn.com/photos/images/newsfeed/...,"[[Myspace, https://knowyourmeme.com/memes/site...",[One of the earliest archived instances of bit...,nan,"[[4chan, https://knowyourmeme.com/memes/sites/...",nan,[https://i.kym-cdn.com/photos/images/newsfeed/...,nan,[Bitches Don\t Know saw a viral takeoff in lat...,nan,nan,[[1] Myspace (via Wayback Machine) – Rich Owen...,"[[Rich Owens, https://web.archive.org/web/2010..."
3,Leave Britney Alone,https://knowyourmeme.com/memes/leave-britney-a...,2021-02-12 11:28:59,https://i.kym-cdn.com/entries/icons/original/0...,2008-12-12 20:11:00,"[youtube, rant, britney spears, response, frea...","[\\leave britney alone\\, \\chris crocker\\]",https://knowyourmeme.com/memes/people/chris-cr...,{Wikipedia: https://en.wikipedia.org/wiki/Chri...,600,315,“Leave Britney Alone” is a viral video in whic...,confirmed,YouTube,2007,nan,[Leave Britney Alone is a viral video in which...,nan,"[[viral video, https://knowyourmeme.com/memes/...","[On September 10, 2007, Chris Crocker uploaded...",nan,"[[MTV Music Aw

In [ ]:
# get the category list from your dataframe
column_names = df.columns
print(column_names)


Index(['Title', 'URL', 'TimeUpdated', 'Image', 'TimeAdded', 'Tags', 'Keywords',
       'Parent', 'References', 'ImageWidth', 'ImageHeight',
       'SocialMediaDescription', 'Status', 'Origin', 'Year', 'Type',
       'AboutText', 'AboutImages', 'AboutLinks', 'OriginText', 'OriginImages',
       'OriginLinks', 'SpreadText', 'SpreadImages', 'SpreadLinks',
       'NotExamplesText', 'NotExamplesImages', 'NotExamplesLinks',
       'SearchIntText', 'SearchIntImages', 'SearchIntLinks', 'ExtRefText',
       'ExtRefLinks'],
      dtype='object')


# Create nodes using py2neo



In [ ]:
## https://neo4j.com/developer/desktop-csv-import/
## https://neo4j.com/labs/apoc/4.1/import/load-csv/

df.to_csv('graph_data.csv',index=False)


In [ ]:
## populate the database
## https://py2neo.org/2021.1/bulk/index.html
## https://neo4j.com/docs/cypher-manual/current/clauses/merge/
## A row is a node.
## A table name is a label name.
## A join or foreign key is a relationship.
!pip install py2neo 
from py2neo.bulk import create_nodes
from py2neo import Node, Graph, Relationship, NodeMatcher,Path
from py2neo.matching import *

     |████████████████████████████████| 177 kB 5.2 MB/s 


In [ ]:
graph_df = pd.read_csv('graph_data.csv',encoding='utf8')
split_df = graph_df


In [ ]:
graph_df = pd.read_csv('graph_data.csv',encoding='utf8')
ref= {}
for index,row in graph_df.iterrows():
  print(row)
  ref[row['Title']]= Node("Ref",
                     reference=str(row['References']))

In [ ]:
origin= {}
for index,row in graph_df.iterrows():
  origin[row['Title']]= Node("Origin",
                        originText=str(row["OriginText"]),
                        originImages=str(row["OriginImages"]),
                        originLinks=str(row["OriginLinks"]))

In [ ]:
keywords= {}
for index,row in graph_df.iterrows():
  keywords[row['Title']]= Node("Keywords",
                          keywords=str(row["Keywords"]))

In [ ]:
parent= {}
for index,row in graph_df.iterrows():
   parent[row['Title']]= Node("Parent",
                         url=str(row["URL"]))

In [ ]:
search= {}
for index,row in graph_df.iterrows():
  keywords[row['Title']]= Node("Search",
                          searchInText=str(row["SearchIntText"]),
                          searchIntImages=str(row["SearchIntImages"]),
                          searchIntLinks=str(row["SearchIntLinks"]))

In [90]:
memes= {}
for index,row in graph_df.iterrows():
  #  print(row)
  memes[row['Title']] = Node("Meme",
                    title=str(row['Title']),
                    timeUpdated=str(row["TimeUpdated"]),
                    image=str(row["Image"]),
                    timeAdded=str(row["TimeAdded"]),
                    tags=str(row["Tags"]),
                    parent=str(row["Parent"]),
                    imageWidth=str(row['ImageWidth']),
                    imageHeight=str(row["ImageHeight"]),
                    socialMediaDescription=str(row["SocialMediaDescription"]),
                    status=str(row["Status"]),
                    origin=str(row["Origin"]),
                    year=str(row["Year"]),
                    type=str(row["Type"]),
                    aboutText=str(row["AboutText"]),
                    aboutImages=str(row["AboutImages"]),
                    aboutLinks=str(row["AboutLinks"]),
                    spreadText=str(row["SpreadText"]),
                    spreadImages=str(row["SpreadImages"]),
                    spreadLinks=str(row["SpreadLinks"]),
                    notExamplesText=str(row["NotExamplesText"]),
                    notExamplesImages=str(row["NotExamplesImages"]),
                    notExampleLinks=str(row["NotExamplesLinks"]),
                    extRefText=str(row["ExtRefText"]),
                    extRefLinks=str(row["ExtRefLinks"]))

# Create nodes and relations

In [ ]:
# use title to retrieve a meme 
print(memes.get("Chocobo Racing Remixes"))


(:Meme {aboutImages: 'nan', aboutLinks: '[[MAD, https://knowyourmeme.com/memes/subcultures/mads], [Nico Nico Douga, https://knowyourmeme.com/memes/sites/nico-nico-douga], [YouTube Poop Music Videos, https://knowyourmeme.com/memes/ytpmvs/], [YouTube, https://knowyourmeme.com/memes/sites/youtube]]', aboutText: '[Chocobo Racing Remixes , also known as X racing, are a series of MAD videos on the Japanese video sharing service Nico Nico Douga (NND) and YouTube Poop Music Videos (YTPMVs) on YouTube.]', extRefLinks: '[[Chocobo Racing, https://en.wikipedia.org/wiki/Chocobo_Racing], [[\u904a\u622f\u738bMAD]\u30ab\u30a4\u30d0\u30ec\u30fc\u30b7\u30f3\u30b0\\u3000\u30a8\u30cd\u30b3\u30f3\u3078\u306e\u300e\u30ed\u30fc\u30c9\u300f, http://www.nicovideo.jp/watch/sm4207541], [\u30b7\u30c9\u306e\u30c6\u30b9\u30c8\u30b3\u30fc\u30b9, http://www.nicovideo.jp/tag/%E3%82%B7%E3%83%89%E3%81%AE%E3%83%86%E3%82%B9%E3%83%88%E3%82%B3%E3%83%BC%E3%82%B9], [\u30c1\u30e7\u30b3\u30dc\u30ec\u30fc\u30b7\u30f3\u30b0(49\u6

In [ ]:
print(memes.get("The Bogeyman"))

(:Meme {aboutImages: 'nan', aboutLinks: '[[creepypasta, https://knowyourmeme.com/memes/creepypasta], [Slender Man, https://knowyourmeme.com/memes/slender-man], [The Rake, https://knowyourmeme.com/memes/the-rake]]', aboutText: '[The Bogeyman (also spelled as bogieman, boogeyman or boogieman) is an imaginary creature commonly used by adults to scare and terrorize children into submissive behaviour by telling them that if they misbehave, the bogeyman will get them. Unlike other creature-based creepypasta such as Slender Man and The Rake, the Bogeyman has no specific appearance in its stories and conceptions about it can vary drastically per nationality or household. Instead he is simply a non-specific embodiment of terror, whos goal depends on what purpose needs serving.]', extRefLinks: '[[Bogey, http://www.etymonline.com/index.php?term=bogey], [Francisco Goya, https://en.wikipedia.org/wiki/Goya], [Bogeyman, http://en.wiktionary.org/wiki/bogeyman], [Bogeyman, http://www.merriam-webster.co

In [ ]:
memeref= Relationship(memes, "REFERED_BY", ref)

In [ ]:
memeparent= Relationship(memes, "CHILD_OF", parent)

In [ ]:
memekeywords= Relationship(memes, "FOUND_BY", keywords)

In [ ]:
memeorigin= Relationship(memes, "ORIGINATES_FROM", origin)

In [ ]:
memesearch= Relationship(memes, "SEARCHED_BY", search)